<a href="https://colab.research.google.com/github/mkmagaya/Machine-Learning-Projects-Python-/blob/master/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import string 
import re
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/R181599G/spam_or_not_spam.csv')

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   2999 non-null   object
 1   label   3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [ ]:
dataset.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [ ]:
dataset.loc[dataset.label == 'Not Spam']=0
dataset.loc[dataset.label == 'Spam']=1

In [ ]:
#split data into x and y variables
X = dataset['email']
y = dataset['label']

In [ ]:
#splitting data into training and test sets
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

Building a vocabulary

In [ ]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Load text data
def load_doc(filename):
# open the file as read only
  file = open(filename, 'r' )
# read all text
  text = file.read()
# close the file
  file.close()
  return text

In [ ]:
from os import listdir


In [ ]:
def process_docs(directory):
  for filename in listdir(directory):
    if not filename.endswith(".txt"):
      next
    path = directory + '/' + filename
    doc = load_doc(path)
    # print( ' Loaded %s ' % filename)

In [ ]:
#Cleaning the doc
def clean_doc(doc):
  tokens = doc.split()
  re_punc = re.compile( '[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub( '' , w) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words( 'english' ))
  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1]
  return tokens


In [ ]:
#add a email review to a vocabulary
filename ='/content/drive/MyDrive/Colab Notebooks/exam/datasets/spam_or_not_spam.csv'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
def add_doc_to_vocab(filename, vocab):
# load doc
  doc = load_doc(filename)
# clean doc
  tokens = clean_doc(doc)
# update counts
  vocab.update(tokens)

In [ ]:
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
  for filename in listdir(directory):
# skip files that do not have the right extension
    if not filename.endswith(".txt"):
      next
# create the full path of the file to open
    path = directory + '/' + filename
# add doc to vocab
    add_doc_to_vocab(path, vocab)

In [ ]:

import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# define vocab
vocab = Counter()
process_docs( '/content/drive/MyDrive/Colab Notebooks/exam/datasets/spam_or_not_spam.csv' , vocab)

# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

33648
[('NUMBER', 33465), ('URL', 7974), ('list', 2171), ('one', 1674), ('get', 1590), ('people', 1315), ('time', 1290), ('use', 1255), ('would', 1254), ('like', 1248), ('new', 1200), ('free', 1180), ('email', 1168), ('mail', 1039), ('wrote', 921), ('also', 914), ('rpm', 896), ('make', 879), ('date', 871), ('mailing', 871), ('message', 854), ('world', 806), ('said', 803), ('us', 798), ('hyperlink', 786), ('spamassassin', 754), ('way', 728), ('money', 722), ('first', 719), ('users', 719), ('could', 718), ('url', 708), ('information', 707), ('think', 699), ('work', 694), ('even', 685), ('spam', 654), ('know', 654), ('want', 641), ('see', 634), ('please', 630), ('may', 611), ('exmh', 593), ('much', 581), ('using', 576), ('well', 576), ('business', 569), ('many', 567), ('linux', 556), ('need', 556)]


In [ ]:
min_occurance = 5
tokens = [k for k,c in vocab.items() if c >= min_occurance]
print(len(tokens))

9297


In [ ]:
def save_list(lines, filename):
  data = '\n' .join(lines)
  file = open(filename, 'w' )
  file.write(data)
  file.close()
# save tokens to a vocabulary file
save_list(tokens, '/content/drive/MyDrive/Colab Notebooks/exam/datasets/spam_or_not_spam.csv' )

In [ ]:
vocab_filename = '/content/drive/MyDrive/Colab Notebooks/exam/datasets/spam_or_not_spam.csv'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)


In [ ]:
def doc_to_line(filename, vocab):
# load the doc
  doc = load_doc(filename)
# clean doc
  tokens = clean_doc(doc)
# filter by vocab
  tokens = [w for w in tokens if w in vocab]
  return ' '.join(tokens)


In [ ]:
def process_docs(directory, vocab):
  lines = list()
  for filename in listdir(directory):
    if not filename.endswith(".txt"):
      next
    path = directory + '/' + filename
    line = doc_to_line(path, vocab)
    lines.append(line)
  return lines

In [ ]:
spam = process_docs( 'question1' , vocab)
save_list(spam, 'spamemails.txt' )


Converting to word embedding vectors

In [ ]:
from gensim.models import Word2Vec

In [ ]:
email = 'vocab.txt'
model = Word2Vec(email, min_count=1)

print(model)

words = list(model.wv.vocab)


Word2Vec(vocab=34, size=100, alpha=0.025)


In [ ]:
print(words)

['/', 'c', 'o', 'n', 't', 'e', 'd', 'r', 'i', 'v', 'M', 'y', ' ', 'D', 'C', 'l', 'a', 'b', 'N', 'k', 's', 'E', 'x', 'm', 'R', '1', '8', '5', '9', '3', 'L', 'q', 'u', '.']


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
input_file = 'vocab.txt'
word2vec_output_file = 'vocab.txt.word2vec'
glove2word2vec(input_file, word2vec_output_file)

(9297, 0)

In [ ]:
#Building LSTM Model 

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [ ]:
def save_doc(lines, filename):
  data = '\n' .join(lines)
  file = open(filename, 'w' )
  file.write(data)
  file.close()

In [ ]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
  # select sequence of tokens
  seq = tokens[i-length:i]

  # convert into a line
  line = ' '.join(seq)

  # store
  sequences.append(line)

print(f'Total Sequences: {len(sequences)}')
# save sequences to file
out_filename = 'vocab.txt'
save_doc(sequences, out_filename)

# load
in_filename = 'vocab.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Total Sequences: 9246


In [ ]:
# load
in_filename = 'vocab.txt'
doc = load_doc(in_filename)
lines = doc.split( '\n' )
lines[:5]

['date wed NUMBER aug chris garrigues cwg dated deepeddy com message id tmda vircio reproduce error like every time without fail debug log pick happening exec inbox list lbrace subject ftp rbrace sequence mercury hit marking hits syntax expression int note run command hand delta comes obviously version nmh using compiled',
 'wed NUMBER aug chris garrigues cwg dated deepeddy com message id tmda vircio reproduce error like every time without fail debug log pick happening exec inbox list lbrace subject ftp rbrace sequence mercury hit marking hits syntax expression int note run command hand delta comes obviously version nmh using compiled URL',
 'NUMBER aug chris garrigues cwg dated deepeddy com message id tmda vircio reproduce error like every time without fail debug log pick happening exec inbox list lbrace subject ftp rbrace sequence mercury hit marking hits syntax expression int note run command hand delta comes obviously version nmh using compiled URL sun',
 'aug chris garrigues cwg d

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(sequences[:1])

[[9292, 9290, 3, 9287, 9285, 9283, 9281, 9279, 9277, 9275, 9273, 9271, 9269, 9267, 9265, 9263, 9261, 9259, 9257, 9255, 9253, 9251, 9249, 9247, 9245, 9243, 9241, 9239, 9237, 9235, 9233, 9231, 9229, 9227, 9225, 9223, 9221, 9219, 9217, 9215, 9213, 9211, 9209, 9207, 9205, 9203, 9201, 9199, 9197, 9195, 4]]


In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
def define_model(vocab_size, seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 50, input_length=seq_length))
  model.add(LSTM(100, return_sequences=True))
  model.add(LSTM(100))
  model.add(Dense(100, activation= 'relu' ))
  model.add(Dense(vocab_size, activation= 'softmax'))
  model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
  model.summary()
  plot_model(model, to_file= '/model.png' , show_shapes=True)
  return model

In [ ]:
model = define_model(vocab_size, seq_length)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            464700    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 9294)              938694    
Total params: 1,554,294
Trainable params: 1,554,294
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, batch_size=10, epochs=2)

Epoch 1/2
925/925 [==============================] - 85s 88ms/step - loss: 9.2938 - accuracy: 0.0000e+00
Epoch 2/2
925/925 [==============================] - 80s 86ms/step - loss: 9.1741 - accuracy: 0.0000e+00


In [ ]:
# save the model to file
model.save('model.h5')
# save the tokenizer
with open('tokenizer.pkl' , 'wb') as token_sv:
  dump(tokenizer, token_sv) 

In [ ]:
#evaluate the model
_, acc = model.evaluate(X, y, verbose=2)
print(f'Train Accuracy: {(acc*100)}')

289/289 - 11s - loss: 9.1572 - accuracy: 1.0815e-04
Train Accuracy: 0.010815487621584907
